#### Import required packages and set the up the caches to be in the correct place

In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["HF_HOME"] = "/.cache/hf"
os.environ["TRANSFORMERS_CACHE"] = "/.cache/transformers"
os.environ["HF_DATASETS_CACHE"] = "/.cache/datasets"
os.environ["HUGGINGFACE_HUB_CACHE"] = "/.cache/hub"
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import json
import tqdm
import pandas as pd
import re

/vol/research/TopDownVideo/hw01558/conda/envs/dis/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/user/HS401/hw01558/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/vol/research/TopDownVideo/hw01558/conda/envs/dis/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


#### Login to hugging face
#### Requires a token that allows access to the model, needs to be requested on hugging face

In [2]:
from huggingface_hub import login
login(token="")


#### Load model

In [ ]:
model_name = "meta-llama/Llama-2-13b-chat-hf" 

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto", 
)

#### Load the stories

In [4]:
file_path = 'NewYorkerStories.json'

with open(file_path, 'r') as f:
    stories = json.load(f)

#### Load the instructions to make the prompts

In [3]:
file_path = 'LanguageAndLitDevices'

with open(file_path, 'r', encoding='utf-8') as f:
    instructions = [line.strip() for line in f]

#### Function that combines instruction and story to create prompt

In [6]:
def build_prompt(story, instruction):
    prompt = f"""    
    You are a creative writing expert. {instruction}

    Story:
    {story}
    """
    return prompt

#### Main program: goes through each instruction and story, takes the response, splits the score and the reasoning, and then saves them

The stories are stored as a list of dictionaries. Each dictionary represents a single story and contains the following keys:

content - the full text of the story

story_id - name of author and number of story (e.g. 5_Claude)

story_name - story title


In [ ]:
#Set up output directory for results
output_dir = "instruction_tests/lang_and_lit/llama_13b/run_0"
os.makedirs(output_dir, exist_ok=True)

#Initialise variables
scores = pd.DataFrame()
i=0

#Loop through each instruction
for instruction in instructions:
    i+=1
    print('-------- Instruction ',i, '-------------')
    #Loop through each story
    for story in stories:
        print('Story: ',story['story_id'])

        #Reset cache and model to ensure space isn't overwhelmed
        torch.cuda.empty_cache()
        model.eval()

        #Build prompt
        prompt = build_prompt(story['content'], instruction)
        
        # Tokenize prompt
        inputs = tokenizer(prompt, return_tensors="pt")
        inputs = inputs.to(model.device) 
        input_token_length = inputs["input_ids"].shape[1]

        #Generate response
        outputs = model.generate(**inputs, max_new_tokens=500, temperature=0.7,)
    
        generated_token_ids = outputs[0][input_token_length:]
        response = tokenizer.decode(generated_token_ids, skip_special_tokens=True)

        # Find the score from in the response
        match = re.search(r'Score:\s*(\d+)', response)
        if match:
            score = int(match.group(1))
        else:
            score = "N/A"

        #Save score
        new_row = pd.DataFrame([{"story": story['story_id'].split("_",1)[0], "writer": story['story_id'].split("_",1)[1], "promt": i, "score":score}])
        scores = pd.concat([scores, new_row], ignore_index=True)
           
        # Save score and response
        output_path = os.path.join(output_dir, f"{story['story_id']}_instruction_{i}.json")
        with open(output_path, "w") as f:
            json.dump({"story_name": story['story_name'],'story_writer':story['story_id'], 'instruction':instruction, "response": response}, f, indent=2)
        
        print(response)
        
#Save all the scores to csv        
scores.to_csv(os.path.join(output_dir,'scores.csv'), index=False)